In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Files
import input_net
import utils

# Packages
from os import path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# Transformers
from transformers import AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer
from transformers import BartConfig, BartForSequenceClassification, BartTokenizer
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from transformers import CamembertConfig, CamembertForSequenceClassification, CamembertTokenizer
from transformers import DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import FlaubertConfig, FlaubertForSequenceClassification, FlaubertTokenizer
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
from transformers import XLMConfig, XLMForSequenceClassification, XLMTokenizer
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer
from sklearn.model_selection import train_test_split

from transformers import AdamW, get_linear_schedule_with_warmup

Using TensorFlow backend.


## Models

In [3]:
#                       Model                                   Tokenizer               Pretrained weights shortcut 
MODEL_CLASSES = {
    'albert': (         AlbertForSequenceClassification,        AlbertTokenizer,        'albert-large-v2'),
    'bart': (           BartForSequenceClassification,          BartTokenizer,          'bart-large'),
    'bert': (           BertForSequenceClassification,          BertTokenizer,          'bert-base-uncased'),
    'camembert': (      CamembertForSequenceClassification,     CamembertTokenizer,     'camembert-base'),
    'distilbert': (     DistilBertForSequenceClassification,    DistilBertTokenizer,    'distilbert-base-uncased'),
    'flaubert': (       FlaubertForSequenceClassification,      FlaubertTokenizer,      'flaubert-base-uncased'),
    'roberta': (        RobertaForSequenceClassification,       RobertaTokenizer,       'roberta-base'),
    'xlm': (            XLMForSequenceClassification,           XLMTokenizer,           'xlm-mlm-en-2048'),
    'xlm_roberta':(     XLMRobertaForSequenceClassification,    XLMRobertaTokenizer,    'xlm-roberta-base'),
    'xlnet': (          XLNetForSequenceClassification,         XLNetTokenizer,         'xlnet-base-cased')
}

args = {
    'model_type': 'bert',
    'do_train': True,
    'do_eval': True,
    'max_seq_length': 70,
    'batch_size': 16, 
    'epochs': 1,
    'learning_rate': 4e-5,
    'num_training_steps': 1000,
    'num_warmup_steps': 100,
    'max_grad_norm': 1.0
}

model_class, tokenizer_class, pretrained_model = MODEL_CLASSES[args['model_type']]

## Input Generation

In [4]:
TRAIN = "data/train.csv"
TEST = "data/test.csv"
INPUT_NET = 'data/input' + str(args['max_seq_length']) + '_' + pretrained_model + '.csv'

if not path.exists(INPUT_NET):
    df = input_net.create_input(TRAIN, INPUT_NET, tokenizer_class, pretrained_model, args)
else:
    df = pd.read_csv(INPUT_NET)

## Net Functions -> quan funcioni tot cridar-ho desde net.py


In [5]:
def batch_generator(data, target, batch_size):
    data = np.array(data)
    target = np.array(target)
    nsamples = len(data)
    perm = np.random.permutation(nsamples)
    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        if target is not None:
            yield data[batch_idx,:], target[batch_idx]
        else:
            yield data[batch_idx], None

def training(model, train_data, train_target, epoch, args):

    criterion = nn.CrossEntropyLoss(reduction='mean')
    optimizer = AdamW(model.parameters(), lr=args['learning_rate'], correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['num_warmup_steps'],
                                                num_training_steps=args['num_training_steps'])
    b = 0
    ncorrect = 0
    total_loss = 0

    batch_size = args['batch_size']
    model.train()
    for X, y in batch_generator(train_data, train_target, batch_size):
        
        X_i, X_s, X_p, y = utils.ToTensor(X,y)
        
        out = model(input_ids=X_i, token_type_ids=X_s, attention_mask=X_p, labels=y)[1]
        
        loss = criterion(out, y)
        loss.backward()
        total_loss += loss
        torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])  # Gradient clipping 
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        out = F.softmax(out, dim=1)

        ncorrect += (torch.max(out, 1)[1] == y).sum().item()
        print("Iteration", b, "of epoch", epoch, "complete", "Loss :", loss.item())
        b += 1
    total_loss /= len(train_data)
    acc = ncorrect/len(train_data) * 100
    return acc, loss

Iteration 100 of epoch 1 complete

def validation(model, eval_data, eval_target, args):
    criterion = nn.CrossEntropyLoss(reduction='mean')

    b = 0
    ncorrect = 0
    total_loss = 0

    batch_size = args['batch_size']
    model.eval()
    for X, y in batch_generator(eval_data, eval_target, batch_size):
        
        X_i, X_s, X_p, y = utils.ToTensor(X,y)
        
        out = model(input_ids=X_i, token_type_ids=X_s, attention_mask=X_p, labels=y)[1]
        
        loss = criterion(out, y)   
        total_loss += loss
        out = F.softmax(out, dim=1)
        ncorrect += (torch.max(out, 1)[1] == y).sum().item()
        print("Batch", b, "-> Validation loss:", loss.item())
        b += 1

    total_loss /= len(eval_data)
    acc = ncorrect/len(eval_data) * 100
    return acc, loss

def build(learn_data, model_class, pretrained_model, args):
    model = model_class.from_pretrained(pretrained_model, num_labels=2)
    print("Model loaded")

    epochs = args['epochs']

    X_train, X_val, y_train, y_val = train_test_split(learn_data.iloc[:,:-1], learn_data.iloc[:,-1],                                                            test_size=0.2, random_state=47)

    train_acc = [None]*epochs
    train_loss = [None]*epochs
    val_acc = [None]*epochs
    val_los = [None]*epochs

    for epoch in range(epochs):
        t_acc, t_loss = training(model, X_train, y_train, epoch, args)
             
        train_acc[epoch] = t_acc
        train_loss[epoch] = t_loss
        
        v_acc, v_loss = validation(model, X_val, y_val, epoch, args)
        val_acc[epoch] = v_acc
        val_loss[epoch] = v_loss

    model.save_pretrained('trained/models/' + pretrained_model)  
    tokenizer.save_pretrained('trained/tokenizers/' + pretrained_model)  

    return train_acc, val_acc

def test(model, test_data, args):
    ncorrect = 0
    total_loss = 0

    batch_size = args['batch_size']

    for X, _ in batch_generator(test_data, batch_size):
        #model.eval()
        X_i, X_s, X_p = utils.ToTensor(X,y=None)
        
        out = model(input_ids=X_i, token_type_ids=X_s, attention_mask=X_p)[0]
        out = F.softmax(out, dim=1)

        ncorrect += (torch.max(out, 1)[1] == y).sum().item()

    acc = ncorrect/len(test_data) * 100
    return acc

## Learning

In [6]:
train_acc, val_acc = build(df, model_class, pretrained_model, args)

Model loaded
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
Batch 0 -> Training loss: 0.809839129447937
Batch 1 -> Training loss: 0.7330045700073242
Batch 2 -> Training loss: 0.7792940139770508
Batch 3 -> Training loss: 0.4851895868778229
Batch 4 -> Training loss: 0.7183745503425598
Batch 5 -> Training loss: 0.9667508006095886
Batch 6 -> Training loss: 0.8012155890464783
Batch 7 -> Training loss: 0.6448269486427307
Batch 8 -> Training loss: 0.867662787437439
Batch 9 -> Training loss: 0.7107551097869873
Batch 10 -> Training loss: 0.3502032160758972
Batch 11 -> Training loss: 0.49457213282585144
Batch 12 -> Training loss: 0.491780549287796
Batch 13 -> Training loss: 0.819337010383606
Batch 14 -> Training loss: 0.6844422817230225
Batch 15 -> Training loss: 0.7478179335594177
Batch 16 -> Training loss: 0.72680

## Proves

In [11]:
model = model_class.from_pretrained(pretrained_model, num_labels=2)
X = np.array(df.iloc[:,:-1])
y = np.array(df.iloc[:,-1])
X_i, X_s, X_p, y = utils.ToTensor(X,y)

# Predict
Predictions = model(input_ids=X_i[:5],token_type_ids=X_s[:5],attention_mask=X_p[:5])
# Training
#Predictions = model(input_ids=X_i[:5],token_type_ids=X_s[:5],attention_mask=X_p[:5], labels=y[:5])